In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [17]:
import pandas as pd
import numpy as np
import runregistry as rr
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters
from tqdm import tqdm

In [3]:
creds = Credentials.from_creds_file()
dials = Dials(creds, workspace="jetmet")

[2025-03-06 16:20:02,719] INFO: Credentials file not found, triggering device authentication flow...
[2025-03-06 16:20:02,815] INFO: This device will expire in 600 seconds.
[2025-03-06 16:20:02,816] INFO: Go to the following url and authenticate: https://auth.cern.ch/auth/realms/cern/device?user_code=GEVC-EOTO
[2025-03-06 16:20:02,817] INFO: Checking authorization status every 5 seconds...
[2025-03-06 16:20:07,915] INFO: Device not authorized yet.
[2025-03-06 16:20:13,008] INFO: Device not authorized yet.
[2025-03-06 16:20:19,888] INFO: Device authorized, authentication finished successfully!


In [15]:
me = "JetMET/MET/pfMETT1/Cleaned/METSig"
runs = [386642, 386640, 385841, 385604, 385422, 385286, 385142, 384243, 384188, 382749, 382213, 380360, 380237, 380195, 379456, 379420]
dataset_regex = "/JetMET0/Run2024.*-PromptReco-v1/DQMIO"

data = []
for run_number in runs:
    res = dials.h1d.list_all(LumisectionHistogram1DFilters(page_size=500, dataset__regex=dataset_regex, me=me,  run_number=run_number), enable_progress=True)
    df = res.to_pandas()
    data.append(df)

data = pd.concat(data)

Progress: 100%|██████████| 2/2 [00:02<00:00,  1.30s/it]


In [16]:
data

,dataset,me,dataset_id,file_id,run_number,ls_number,me_id,x_min,x_max,x_bin,entries,data
0,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674077,386642,1,5,0.0,51.0,51.0,4823,"[1940.0, 1987.0, 670.0, 181.0, 31.0, 8.0, 4.0,..."
1,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674237,386642,2,5,0.0,51.0,51.0,4681,"[1869.0, 1872.0, 714.0, 184.0, 26.0, 6.0, 6.0,..."
2,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674237,386642,3,5,0.0,51.0,51.0,4645,"[1819.0, 1924.0, 672.0, 188.0, 27.0, 8.0, 2.0,..."
3,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674037,386642,4,5,0.0,51.0,51.0,3427,"[1400.0, 1366.0, 517.0, 106.0, 24.0, 7.0, 2.0,..."
4,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674037,386642,5,5,0.0,51.0,51.0,4475,"[1789.0, 1819.0, 660.0, 165.0, 28.0, 8.0, 3.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
550,/JetMET0/Run2024C-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,14949683,14187077757,379420,551,5,0.0,51.0,51.0,2170,"[972.0, 757.0, 277.0, 101.0, 32.0, 16.0, 7.0, ..."
551,/JetMET0/Run2024C-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,14949683,14187077797,379420,552,5,0.0,51.0,51.0,2167,"[987.0, 745.0, 264.0, 117.0, 24.0, 17.0, 6.0, ..."
552,/JetMET0/Run2024C-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,14949683,14187077797,379420,553,5,0.0,51.0,51.0,2138,"[978.0, 794.0, 201.0, 119.0, 27.0, 14.0, 4.0, ..."
553,/JetMET0/Run2024C-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,14949683,14187077757,379420,554,5,0.0,51.0,51.0,2069,"[924.0, 743.0, 234.0, 120.0, 31.0, 6.0, 2.0, 4..."


In [18]:
rr_dataset = "/PromptReco/Collisions2024/DQM"
lumisections = []
for run_number in tqdm(runs):
    joint_ranges = rr.get_joint_lumisection_ranges(run_number, rr_dataset)
    run_lumisections = []
    for joint_range in joint_ranges:
        # Cleanup extra keys in rr lumis, since we only care about the status key
        for key, value in joint_range.items():
            if isinstance(value, dict) and "status" in value.keys():
                joint_range[key] = value["status"] == "GOOD"

        # Repeat the flags for the same range
        start_ls = joint_range.pop("start")
        end_ls = joint_range.pop("end")
        for ls_number in range(start_ls, end_ls + 1):
            run_lumisections.append({"run_number": run_number, "ls_number": ls_number, **joint_range})

    run_lumisections = pd.DataFrame(run_lumisections)
    lumisections.append(run_lumisections)

lumisections = pd.concat(lumisections)

100%|██████████| 16/16 [00:16<00:00,  1.04s/it]


In [20]:
lumis = lumisections[
    lumisections.cms_active &
    lumisections.beam1_stable &
    lumisections.beam2_stable &
    lumisections.beam1_present &
    lumisections.beam2_present &
    lumisections.bpix_ready &
    lumisections.fpix_ready &
    lumisections.tibtid_ready &
    lumisections.tecm_ready &
    lumisections.tecp_ready &
    lumisections.tob_ready &
    lumisections.hbhea_ready &
    lumisections.hbheb_ready &
    lumisections.hbhec_ready &
    lumisections.hf_ready &
    lumisections.ho_ready
]
lumis = lumis[["run_number", "ls_number", "jetmet-jetmet"]]

lumis

,run_number,ls_number,jetmet-jetmet
0,386642,1,True
1,386642,2,True
2,386642,3,True
3,386642,4,True
4,386642,5,True
...,...,...,...
549,379420,550,True
550,379420,551,True
551,379420,552,True
552,379420,553,True


In [21]:
joint_data = data.merge(lumis, on=['run_number', 'ls_number'])
joint_data

,dataset,me,dataset_id,file_id,run_number,ls_number,me_id,x_min,x_max,x_bin,entries,data,jetmet-jetmet
0,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674077,386642,1,5,0.0,51.0,51.0,4823,"[1940.0, 1987.0, 670.0, 181.0, 31.0, 8.0, 4.0,...",True
1,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674237,386642,2,5,0.0,51.0,51.0,4681,"[1869.0, 1872.0, 714.0, 184.0, 26.0, 6.0, 6.0,...",True
2,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674237,386642,3,5,0.0,51.0,51.0,4645,"[1819.0, 1924.0, 672.0, 188.0, 27.0, 8.0, 2.0,...",True
3,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674037,386642,4,5,0.0,51.0,51.0,3427,"[1400.0, 1366.0, 517.0, 106.0, 24.0, 7.0, 2.0,...",True
4,/JetMET0/Run2024I-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,15090031,17813674037,386642,5,5,0.0,51.0,51.0,4475,"[1789.0, 1819.0, 660.0, 165.0, 28.0, 8.0, 3.0,...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13792,/JetMET0/Run2024C-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,14949683,14187077757,379420,550,5,0.0,51.0,51.0,2228,"[1024.0, 767.0, 287.0, 100.0, 31.0, 13.0, 3.0,...",True
13793,/JetMET0/Run2024C-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,14949683,14187077757,379420,551,5,0.0,51.0,51.0,2170,"[972.0, 757.0, 277.0, 101.0, 32.0, 16.0, 7.0, ...",True
13794,/JetMET0/Run2024C-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,14949683,14187077797,379420,552,5,0.0,51.0,51.0,2167,"[987.0, 745.0, 264.0, 117.0, 24.0, 17.0, 6.0, ...",True
13795,/JetMET0/Run2024C-PromptReco-v1/DQMIO,JetMET/MET/pfMETT1/Cleaned/METSig,14949683,14187077797,379420,553,5,0.0,51.0,51.0,2138,"[978.0, 794.0, 201.0, 119.0, 27.0, 14.0, 4.0, ...",True


In [22]:
train_data = joint_data[["data", "jetmet-jetmet"]]
train_data = train_data.rename(columns={'jetmet-jetmet': 'label'})
train_data

,data,label
0,"[1940.0, 1987.0, 670.0, 181.0, 31.0, 8.0, 4.0,...",True
1,"[1869.0, 1872.0, 714.0, 184.0, 26.0, 6.0, 6.0,...",True
2,"[1819.0, 1924.0, 672.0, 188.0, 27.0, 8.0, 2.0,...",True
3,"[1400.0, 1366.0, 517.0, 106.0, 24.0, 7.0, 2.0,...",True
4,"[1789.0, 1819.0, 660.0, 165.0, 28.0, 8.0, 3.0,...",True
...,...,...
13792,"[1024.0, 767.0, 287.0, 100.0, 31.0, 13.0, 3.0,...",True
13793,"[972.0, 757.0, 277.0, 101.0, 32.0, 16.0, 7.0, ...",True
13794,"[987.0, 745.0, 264.0, 117.0, 24.0, 17.0, 6.0, ...",True
13795,"[978.0, 794.0, 201.0, 119.0, 27.0, 14.0, 4.0, ...",True


In [23]:
data_arr = np.vstack(train_data.data)
data_arr

array([[1940., 1987.,  670., ...,    0.,    0.,    0.],
       [1869., 1872.,  714., ...,    0.,    0.,    0.],
       [1819., 1924.,  672., ...,    0.,    0.,    0.],
       ...,
       [ 987.,  745.,  264., ...,    0.,    0.,    0.],
       [ 978.,  794.,  201., ...,    0.,    0.,    0.],
       [ 924.,  743.,  234., ...,    0.,    0.,    0.]])

In [24]:
label_arr = train_data.label.to_numpy().astype(np.float32)
label_arr

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [25]:
unique, counts = np.unique(label_arr, return_counts=True)
dict(zip(unique, counts))

{np.float32(0.0): np.int64(2422), np.float32(1.0): np.int64(11375)}

In [26]:
os.makedirs("../data", exist_ok=True)
np.save(f"../data/data.npy", data_arr)
np.save(f"../data/label.npy", label_arr)